# Using the ASE interface of XTP

## Introduction

This tutorial explains how to perform calculation to predict electronic excitation using the **GWBSE** method. See [the GW Compendium: A Practical Guide to Theoretical Photoemission Spectroscopy](https://doi.org/10.3389/fchem.2019.00377), for an excellent introduction to the method.

## Requirements
* You will need to install **VOTCA** using the instructions described [here](https://github.com/votca/votca/blob/master/share/doc/INSTALL.rst)
* Once the installation is completed you need to activate the VOTCA enviroment by running the `VOTCARC.bash` script that has been installed at the bin subfolder for the path that you have provided for the installation step above
* You need to make sure that the `pyxtp` python module is accessible in your environment

## DFT + GWBSE Calculation of CH4
To run a DFT-GWBSE calculation we will use the python binding to the [xtp_tools](https://www.votca.org/xtp/xtp_tools_overview.html) calculator. We will use this calculator together with the `Atoms` class provided by ASE 

In [ ]:
from ase import Atoms
from pyxtp import xtp

The ASE interface allows to quickly create a CH4 through the library of commonly used molecule. We can then simply define and attach the `xtp` calculator to it. Many options can be passed during the creation of the `xtp` calculator. We here simply specify the number of threads that will be used by the calculator

In [ ]:
atoms = Atoms('CH4')
calc = xtp(nthreads=2)
atoms.calc = calc

### Running a calculation with the default options
To run a DFT-GWBSE calculation using the default options of XTP we can simply ask the calculator to compute for example the potential energy of the system

In [ ]:
atoms.get_potential_energy()

The previous command will run the DFT-GWBSE calculation using the aforementioned defaults. The results are store in the *Current Work Directory* in a file named `methane_summary.xml`. The results are also directly stored in the `atoms` object in a dictionnary that can be accessed via:

In [ ]:
print(atoms.results)

### Modifying the input options
The options of the calculations are specified in a data structure of the calculator called `options`. This data structure is a simple representation of the several XML files that are required to run XTP through the command line interface. You can navigate the data strucrure with its autocomplete functionality, i.e. by pressing the `tab` key. By default the value of these options is set to an empty string, e.g.

In [ ]:
atoms.calc.options.dftpackage.basisset

If leave as is, each option will be set to its default value before the calculation. A new value can easily be assigned to each option, for example to change to the basis set an the functional of the DFT calculation one can do:

In [ ]:
atoms.calc.options.dftpackage.basisset = 'def2-svp'
atoms.calc.options.dftpackage.functional = 'PBE'

The help function can be used to obtain information regarding the different structures within the options. For example

In [ ]:
print(help(atoms.calc.options.dftpackage))

Frequently used options can also be set when creating the calculator. This is the case of the basis set, XC functional and charge. Hence the creating a specific `xtp` calculator via:

In [ ]:
calc = xtp(basis='def2-svp', xc='PBE', charge=1)

will create a calculator using this particular values of the basis set, functional and charge on the molecule.

Once the options are set the calculation can be run as before:

In [ ]:
atoms.get_potential_energy()

### Logging file
By default the log of the calculation is printed on screen. The log file can be redirected to a dedicated file via the option `options.logging_file`, as in:

In [ ]:
atoms.calc.options.logging_file = 'CH4.log'

### Visualizing the results
The results of the calculation can be easily visualized via the `Visualization` class of the `pyxtp` module. For example to plot the absopriton spectrum of the molecule one can use: 

In [ ]:
from pyxtp import Visualization
viz = Visualization(atoms, save_figure=False)
viz.plot_absorption_gaussian()

One can also visualize the quasi particule correction energy via

In [ ]:
viz.plot_qp_corrections()

## Geometry Optimization
The geometry of the molecule can be optimized using either the native functionalities of `XTP` or using `ASE`.

### ASE optimizers
Having a dedicated `ASE` calculator allows to leverage the possibilities offered by the `ASE` ecosystem. For example the geometry optimization of the carbon monoxyde molecule can be done following:  

In [ ]:
from pyxtp import xtp
from ase.io import write
from ase.build import molecule
from ase.optimize import QuasiNewton

atoms = molecule('CO')
atoms.rattle()

calc = xtp(nthreads=2)
calc.select_force(energy='singlets', level=0, dynamic=False)
atoms.calc = calc

dyn = QuasiNewton(atoms, trajectory='test.traj')
dyn.run(fmax=0.01)
write('final.xyz', atoms)

This allows to optimize the geometry to minimize the energy of the lowest singlet state. This is done here via the `select_force` method of the `xtp` calculator. The final geometry is stored in the `final.xyz` file.

### Native XTP optimizer
XTP is equiped with its own geometry optimization that can be activated by setting any of the fields relative to geometry optimization in the `options` data structure.  

In [ ]:
from pyxtp import xtp
from ase.build import molecule

atoms = molecule('CO')
atoms.rattle()

calc = xtp(nthreads=2)
calc.options.geometry_optimization.state = 's1'
calc.options.geometry_optimization.maxiter = 1

atoms.calc = calc
atoms.calc.calculate()